In [1]:
import numpy as np
from matplotlib import pyplot as plt
from dsapol import dsapol
from scipy.signal import correlate
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.signal import fftconvolve
from scipy.ndimage import convolve1d
from scipy.signal import peak_widths
from scipy.stats import chi
from scipy.stats import norm
import panel as pn
pn.extension()
#from numpy.ma import masked_array as ma
import numpy.ma as ma
from scipy.stats import kstest
from scipy.optimize import curve_fit
import time
#import numpy.ma as ma 
%load_ext autoreload
%autoreload 2 

from scipy.signal import find_peaks
from scipy.signal import peak_widths
import copy
import numpy as np
import os
import param

from sigpyproc import FilReader
from sigpyproc.Filterbank import FilterbankBlock
from sigpyproc.Header import Header
from matplotlib import pyplot as plt
import pylab
import pickle
import json
from scipy.interpolate import interp1d
from scipy.stats import chi2
from scipy.stats import chi
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.ndimage import convolve1d
from RMtools_1D.do_RMsynth_1D import run_rmsynth
from RMtools_1D.do_RMclean_1D import run_rmclean
from RMtools_1D.do_QUfit_1D_mnest import run_qufit

plt.rcParams.update({
                    'font.size': 16,
                    'font.family': 'serif',
                    'axes.labelsize': 16,
                    'axes.titlesize': 16,
                    'xtick.labelsize': 12,
                    'ytick.labelsize': 12,
                    'xtick.direction': 'in',
                    'ytick.direction': 'in',
                    'xtick.top': True,
                    'ytick.right': True,
                    'lines.linewidth': 1,
                    'lines.markersize': 5,
                    'legend.fontsize': 14,
                    'legend.borderaxespad': 0,
                    'legend.frameon': False,
                    'legend.loc': 'lower right'})


from astropy.time import Time
from astropy.coordinates import EarthLocation
import astropy.units as u


def gauss_scint(x,bw,amp,off):
    return off + amp*np.exp(-np.log(2)*((x/bw)**2))


def lorentz_scint(x,bw,amp,off):
    return off + amp*(bw/(x**2 + (0.5*bw**2)))

fsize=30
fsize2=20
plt.rcParams.update({
                    'font.size': fsize,
                    'font.family': 'sans-serif',
                    'axes.labelsize': fsize,
                    'axes.titlesize': fsize,
                    'xtick.labelsize': fsize,
                    'ytick.labelsize': fsize,
                    'xtick.direction': 'in',
                    'ytick.direction': 'in',
                    'xtick.top': True,
                    'ytick.right': True,
                    'lines.linewidth': 1,
                    'lines.markersize': 5,
                    'legend.fontsize': fsize2,
                    'legend.borderaxespad': 0,
                    'legend.frameon': False,
                    'legend.loc': 'lower right'})

In [2]:
#import functions from polarization modules
from pol_panel_funcs import pol_plot
from pol_panel_funcs import pol_panel
from pol_panel_funcs import RM_plot
from pol_panel_funcs import RM_panel
from pol_panel_callbacks import callback
from pol_panel_callbacks import callback_cal
from pol_panel_callbacks import callback_link
from pol_panel_callbacks import callback_linkback
from pol_panel_callbacks import callback_RMcal

#import functions from usage module
import usage_panel_funcs

# Try again, implementing based on Casey's Template

In [3]:



def build_pol_tab():
    
    
    
    # make dropdown box to choose FRB file
    x=os.listdir("/media/ubuntu/ssd/sherman/scratch_weights_update_2022-06-03_32-7us") 
    FRB_list = []
    for i in range(len(x)):
        if len(x[i]) >= 10 and x[i][10] == "_":
            FRB_list.append(x[i])

    menu_calplots = pn.widgets.MenuButton(name="Select FRB to Load",
                                          items=FRB_list,
                                         width=200)
    
    load_button = pn.widgets.Button(name='Load FRB', width=100, button_type='primary')
    
    # make dropdown box to choose calibration file
    y = os.listdir("/media/ubuntu/ssd/sherman/code")
    CAL_list = []
    
    for i in range(len(y)):
        if ("POLCAL_PARAMETERS" in y[i]) and ("csv" in y[i]):
            CAL_list.append(y[i])
    calmenu_calplots = pn.widgets.MenuButton(name="Select Calibration File to Load",
                                      items=CAL_list,
                                     width=200)
    
    #calibration button and beam, RA, dec input for PA calibration
    cal_button = pn.widgets.Button(name='Calibrate', width=100, button_type='primary')
    ibeam_input = pn.widgets.IntSlider(name='ibeam', start=0, end=250, step=1, value=0)
    RA_input = pn.widgets.FloatInput(name='RA (degrees)', value=0, step=1e-2, start=0, end=360)
    DEC_input = pn.widgets.FloatInput(name='DEC (degrees)', value=0, step=1e-2, start=-90, end=90)
    

    #create polarization pane and links for initialization
    pane_test = pn.pane.Markdown('# DSA-110 Polarization Dashboard')
    pan1 = pol_panel()
    
    menu_calplots.link(pan1, clicked='frb_name')
    load_button.link(pan1,callbacks={'clicks': callback})#pan1, clicks='frb_submitted')
    
    calmenu_calplots.link(pan1,clicked='cal_name')
    cal_button.link(pan1,callbacks={'clicks': callback_cal})#pan1, clicks='frb_submitted')
    ibeam_input.link(pan1,value='ibeam')
    RA_input.link(pan1,value='RA')
    DEC_input.link(pan1,value='DEC')
    
    #create RM pane and links for passing data
    pan2 = RM_panel()
  
    link_button = pn.widgets.Button(name='Proceed to RM Synthesis',width=200,button_type='primary')
    #link_button.link(pan2,callbacks={'clicks': callback_link})#pan1, clicks='frb_submitted')
    link_button.link(pan2,callbacks={'clicks': lambda target,event: callback_link(target,event,pan1)})
    
    linkback_button = pn.widgets.Button(name='Return to Pol Analysis',width=200,button_type='primary')
    #linkback_button.link(pan1,callbacks={'clicks':callback_linkback})
    linkback_button.link(pan1,callbacks={'clicks':lambda target,event: callback_linkback(target,event,pan2)})
    
 
    RMcal_button = pn.widgets.Button(name='RM Calibrate',width=200,button_type='primary')
    #RMcal_button.link(pan1,callbacks={'clicks':callback_RMcal})
    RMcal_button.link(pan1,callbacks={'clicks':lambda target,event: callback_RMcal(target,event,pan2)})
    
    
    return pn.Column(pane_test,pn.Row(menu_calplots,load_button,calmenu_calplots,ibeam_input,RA_input,DEC_input,cal_button,link_button,linkback_button,RMcal_button),pn.Row(pan1.param,pan1.view,pan2.param,pan2.view))


#ADD JAKOB'S CODE HERE FOR BURSTFIT TAB
def burstfit_tab():
    pass
    
    
#usage tab

def build_usage_tab():
    
    #dedisp tab
    dedisp_usage = """
    ### **Dedispersion Tab Usage**
    
    *Insert dedispersion tab usage here*
    """
    
    #pol tab
    pol_usage = usage_panel_funcs.pol_usage_str
    
    
    #burstfit tab
    burstfit_usage = """
    ### **Burstfit Tab Usage**
    
    *Insert burstfit tab usage here*
    """
    
    
    width=1000
    
    title = """
            # **Burst Analysis Dashboard Usage**
                                      
            ## This page contains instructions and examples for using the Burst Analysis Dashboard
            """
    
    return pn.Column(pn.pane.Markdown(title,width=width),
                     pn.pane.Markdown(dedisp_usage,width=width),
                    pn.pane.Markdown(pol_usage,width=width),
                    pn.pane.Markdown(burstfit_usage,width=width))

In [4]:
pol_pane = pn.param.ParamFunction(build_pol_tab,lazy=True)


In [5]:
usage_pane = pn.param.ParamFunction(build_usage_tab,lazy=True)

In [6]:
dashboard = pn.Column(pn.Row(pn.layout.tabs.Tabs(('Usage', usage_pane),
                                                 ('Polarization', pol_pane),
                                                 #('Burstfit',burstfit_tab),
                                                dynamic=True, active=0),
                             margin=(10, 10, 10, 10), background='#f0f0f0'))

In [7]:
dashboard

Column
    [0] Row(background='#f0f0f0', margin=(10, 10, 10, 10))
        [0] Tabs(dynamic=True)
            [0] ParamFunction(function, lazy=True)
            [1] ParamFunction(function, lazy=True)

In [54]:
dashboard.servable(title="Burst Analysis Dashboard")

Column
    [0] Row(background='#f0f0f0', margin=(10, 10, 10, 10))
        [0] Tabs(dynamic=True)
            [0] ParamFunction(function, lazy=True)
            [1] ParamFunction(function, lazy=True)